# Solving Semidefinite Programs (SDPs) in Python

In this notebook, we will be looking at how one may specify and numerically solve SDPs in Python. 

You may already be aware of the CVX software package for MATLAB. While this software is powerful and useful for solving a wide variety of SDPs, it requires a MATLAB license which can be cost prohibitive. 

The `cvxpy` module in Python is an open-source option for solving convex optimization problems. The way in which one specifies an SDP in Python can be a bit awkward if you have not seen it before. In this notebook, we will cover how to specify and solve SDPs using this module. 

## Semidefinite programming

- Generalization of linear programming.
- Powerful tool with many applications in quantum information.
- SDPs are efficiently solvable in (polynomial time).
- Software packages for solving SDPs exist (`cvxpy`).

## The "standard form" for SDPs

$$
            \begin{equation}
                \begin{aligned}
                    \underline{\text{Primal problem:}} \quad & \\
                    \text{maximize:} \quad & \langle A, X \rangle \\
                    \text{subject to:} \quad & \Phi(X) = B, \\
                                       \quad & X \in \text{Pos}(\mathcal{X}).
                \end{aligned}
            \end{equation}
$$
$$
            \begin{equation}
                \begin{aligned}
                    \underline{\text{Dual problem:}} \quad & \\
                    \text{minimize:} \quad & \langle B, Y \rangle \\
                    \text{subject to:} \quad & \Phi^*(Y) \geq A, \\
                                       \quad & Y \in \text{Herm}(\mathcal{Y}).
                \end{aligned}
            \end{equation}
$$

## Installing the `cvxpy` and `numpy` modules

In [ ]:
pip install cvxpy

In [ ]:
pip install numpy

# Simple examples

Let's take a look at a simple SDP to see how we can encode it using Python and `cvxpy`.

$$
            \begin{equation}
                \begin{aligned}
                    \underline{\text{Primal problem:}} \quad & \\
                    \text{maximize:} \quad & \langle \mathbb{I}_4, X \rangle \\
                    \text{subject to:} \quad & X = \mathbb{I}_4, \\
                                       \quad & X \in \text{Pos}(\mathcal{X}).
                \end{aligned}
            \end{equation}
$$

In [19]:
import cvxpy
import numpy as np

# Declare a 4x4 variable for our SDP.
x_var = cvxpy.Variable((4, 4))

# We can specify the constraints for our SDP in a Python list.
constraints = []

# We can write the inner product:
#    <A, X> == Tr(A^*, X)
objective = cvxpy.Maximize(cvxpy.trace(x_var.H @ np.identity(4)))
constraints.append(x_var == np.identity(4))
constraints.append(x_var >> 0)

# Feed in the objective and constraints to the solver.
problem = cvxpy.Problem(objective, constraints)
val = problem.solve()

# Print out the optimal value along with the 
print(val)
print(x_var.value)

3.9999999999999996
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


# Computing the fidelity function

Problem: Calculate the fidelity of two quantum states $\rho$ and $\sigma$ defined as $F(\rho, \sigma) = ||\sqrt{\rho}\sqrt{\sigma}||_1$.

$$
            \begin{equation}
                \begin{aligned}
                    \underline{\text{Primal problem:}} \quad & \\
                    \text{maximize:} \quad & \frac{1}{2} \text{Tr}(X) + \frac{1}{2} \text{Tr}(X^*) \\
                    \text{subject to:} \quad & \begin{pmatrix}
                                                   \rho & X \\
                                                   X^* & \sigma
                                               \end{pmatrix} \geq 0
                \end{aligned}
            \end{equation}
$$

In [ ]:
        z_var = cvxpy.Variable(rho.shape, complex=True)
        objective = cvxpy.Maximize(cvxpy.real(cvxpy.trace(z_var + z_var.H)))
        constraints = [cvxpy.bmat([[rho, z_var], [z_var.H, sigma]]) >> 0]
        problem = cvxpy.Problem(objective, constraints)

        return 1 / 2 * problem.solve()